# Uji Kelayakan kedua model

In [8]:
import pandas as pd
import numpy as np
import joblib
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def generate_neutral_and_minority_data():
    """
    Membuat dua set data: satu set netral dan satu set khusus minoritas (PhD).
    """
    # 1. Data Uji Netral (Profil Umum & Bervariasi)
    neutral_df = pd.DataFrame({
        'Age': [24, 35, 48, 29, 52, 22, 41, 33],
        'Gender': ['Female', 'Male', 'Male', 'Female', 'Male', 'Male', 'Female', 'Male'],
        'Department': ['Marketing', 'Engineering', 'Sales', 'Product', 'Finance', 'Engineering', 'HR', 'Product'],
        'Job_Title': ['Analyst', 'Engineer', 'Manager', 'Engineer', 'Executive', 'Intern', 'Manager', 'Manager'],
        'Experience_Years': [2, 12, 22, 6, 28, 0, 18, 10],
        'Education_Level': ['Bachelor', 'Master', 'Bachelor', 'Bachelor', 'Master', 'Bachelor', 'PhD', 'Master'],
        'Location': ['Chicago', 'Austin', 'New York', 'San Francisco', 'Seattle', 'New York', 'Chicago', 'Austin']
    })
    
    # 2. Data Uji Minoritas (Profil Khusus PhD)
    minority_df = pd.DataFrame({
        'Age': [28, 35, 45, 55, 32, 50, 48],
        'Gender': ['Female', 'Male', 'Male', 'Female', 'Male', 'Male', 'Female'],
        'Department': ['Engineering', 'Product', 'Sales', 'HR', 'Marketing', 'Finance', 'Engineering'],
        'Job_Title': ['Engineer', 'Manager', 'Executive', 'Manager', 'Analyst', 'Executive', 'Intern'],
        'Experience_Years': [4, 10, 20, 30, 8, 25, 5],
        'Education_Level': ['PhD', 'PhD', 'PhD', 'PhD', 'PhD', 'PhD', 'PhD'],
        'Location': ['Austin', 'San Francisco', 'New York', 'Chicago', 'Seattle', 'New York', 'Austin']
    })
    
    return neutral_df, minority_df

def run_comparison_test(test_df, description=""):
    """
    Menjalankan prediksi pada data uji dan menampilkan tabel perbandingan.
    """
    print("\n" + "="*70)
    print(f" HASIL PENGUJIAN PADA: {description}")
    print("="*70)
    
    try:
        model_anda = joblib.load('../models/model_setelah_tunninng_v1.pkl')
        model_saran = joblib.load('../models/model_improve_terbaru_v2.pkl')

        # Siapkan data untuk model Anda
        test_df_for_model_anda = test_df.copy()
        test_df_for_model_anda["Gender"] = test_df_for_model_anda["Gender"].map({"Female": 0, "Male": 1})
        test_df_for_model_anda["Education_Level"] = test_df_for_model_anda["Education_Level"].map({"Bachelor": 0, "Master": 1, "PhD": 3})
        
        # Prediksi
        prediksi_anda = model_anda.predict(test_df_for_model_anda)
        prediksi_saran = model_saran.predict(test_df)
        
        # Buat DataFrame hasil
        results_df = test_df.copy()
        results_df['Prediksi_Anda (USD)'] = [f"${gaji:,.0f}" for gaji in prediksi_anda]
        results_df['Prediksi_Saran (USD)'] = [f"${gaji:,.0f}" for gaji in prediksi_saran]
        results_df['Selisih_Absolut'] = np.abs(prediksi_anda - prediksi_saran)
        
        display(HTML(results_df.to_html(index=False)))
        
    except Exception as e:
        print(f"Terjadi error saat menjalankan tes: {e}")

def check_overfitting():
    """
    Memuat data asli, melatih ulang, dan membandingkan skor train vs. test.
    """
    print("\n" + "="*70)
    print(" ANALISIS OVERFITTING MODEL")
    print("="*70)
    print("Membandingkan skor R² pada data yang dilihat (Train) vs. data baru (Test).\n")
    
    try:
        # Memuat dan membagi data asli seperti di notebook awal
        df_original = pd.read_csv("../../data/Employers_data.csv")
        df_cleaned = df_original.drop(columns=['Employee_ID', 'Name'])
        
        # Encoding manual untuk Model Anda
        df_anda = df_cleaned.copy()
        df_anda["Gender"] = df_anda["Gender"].map({"Female": 0, "Male": 1})
        df_anda["Education_Level"] = df_anda["Education_Level"].map({"Bachelor": 0, "Master": 1, "PhD": 3})
        X_anda = df_anda.drop('Salary', axis=1)
        y_anda = df_anda['Salary']
        X_train_anda, X_test_anda, y_train_anda, y_test_anda = train_test_split(X_anda, y_anda, test_size=0.2, random_state=42)

        # Tanpa encoding manual untuk Model Saran
        X_saran = df_cleaned.drop('Salary', axis=1)
        y_saran = df_cleaned['Salary']
        X_train_saran, X_test_saran, y_train_saran, y_test_saran = train_test_split(X_saran, y_saran, test_size=0.2, random_state=42)

        # Memuat model
        model_anda = joblib.load('../models/model_setelah_tunninng_v1.pkl')
        model_saran = joblib.load('../models/model_improve_terbaru_v2.pkl')

        # Evaluasi Model Anda
        score_train_anda = r2_score(y_train_anda, model_anda.predict(X_train_anda))
        score_test_anda = r2_score(y_test_anda, model_anda.predict(X_test_anda))
        
        # Evaluasi Model Saran
        score_train_saran = r2_score(y_train_saran, model_saran.predict(X_train_saran))
        score_test_saran = r2_score(y_test_saran, model_saran.predict(X_test_saran))

        # Tampilkan Laporan Overfitting
        report = f"""
        Model Anda (model_setelah_tunninng_v1.pkl):
        -------------------------------------------
        Skor Data Latih (Train) : {score_train_anda:.4f}
        Skor Data Uji (Test)    : {score_test_anda:.4f}
        Selisih (Train - Test)  : {score_train_anda - score_test_anda:.4f}

        Model Saran (model_improve_terbaru_v2.pkl):
        -------------------------------------------
        Skor Data Latih (Train) : {score_train_saran:.4f}
        Skor Data Uji (Test)    : {score_test_saran:.4f}
        Selisih (Train - Test)  : {score_train_saran - score_test_saran:.4f}
        """
        print(report)
        print("Analisis: Selisih yang kecil (< 0.05) menunjukkan model dapat menggeneralisasi dengan baik (tidak overfit).")
        print("="*70)

    except Exception as e:
        print(f"Gagal melakukan cek overfitting: {e}")


# --- EKSEKUSI SEMUA PENGUJIAN ---

# 1. Membuat data uji
data_netral, data_minoritas_phd = generate_neutral_and_minority_data()

# 2. Menjalankan tes pada data netral
run_comparison_test(data_netral, description="Data Uji Netral")

# 3. Menjalankan tes pada data minoritas
run_comparison_test(data_minoritas_phd, description="Data Uji Minoritas (PhD)")

# 4. Mengecek overfitting
check_overfitting()


 HASIL PENGUJIAN PADA: Data Uji Netral


Age,Gender,Department,Job_Title,Experience_Years,Education_Level,Location,Prediksi_Anda (USD),Prediksi_Saran (USD),Selisih_Absolut
24,Female,Marketing,Analyst,2,Bachelor,Chicago,"$64,415","$63,581",833.094299
35,Male,Engineering,Engineer,12,Master,Austin,"$107,690","$109,326",1636.535974
48,Male,Sales,Manager,22,Bachelor,New York,"$137,196","$136,276",919.623658
29,Female,Product,Engineer,6,Bachelor,San Francisco,"$93,861","$93,230",631.042781
52,Male,Finance,Executive,28,Master,Seattle,"$190,648","$191,074",425.944498
22,Male,Engineering,Intern,0,Bachelor,New York,"$35,877","$35,146",730.456104
41,Female,HR,Manager,18,PhD,Chicago,"$138,224","$138,578",353.909622
33,Male,Product,Manager,10,Master,Austin,"$123,783","$123,182",600.916859



 HASIL PENGUJIAN PADA: Data Uji Minoritas (PhD)


Age,Gender,Department,Job_Title,Experience_Years,Education_Level,Location,Prediksi_Anda (USD),Prediksi_Saran (USD),Selisih_Absolut
28,Female,Engineering,Engineer,4,PhD,Austin,"$94,859","$94,781",77.855048
35,Male,Product,Manager,10,PhD,San Francisco,"$125,506","$124,993",513.176387
45,Male,Sales,Executive,20,PhD,New York,"$179,734","$179,236",498.422592
55,Female,HR,Manager,30,PhD,Chicago,"$160,433","$164,939",4506.598242
32,Male,Marketing,Analyst,8,PhD,Seattle,"$98,314","$91,713",6601.181606
50,Male,Finance,Executive,25,PhD,New York,"$187,644","$187,008",636.302973
48,Female,Engineering,Intern,5,PhD,Austin,"$105,837","$108,173",2335.879870



 ANALISIS OVERFITTING MODEL
Membandingkan skor R² pada data yang dilihat (Train) vs. data baru (Test).


        Model Anda (model_setelah_tunninng_v1.pkl):
        -------------------------------------------
        Skor Data Latih (Train) : 0.9930
        Skor Data Uji (Test)    : 0.9910
        Selisih (Train - Test)  : 0.0020

        Model Saran (model_improve_terbaru_v2.pkl):
        -------------------------------------------
        Skor Data Latih (Train) : 0.9921
        Skor Data Uji (Test)    : 0.9924
        Selisih (Train - Test)  : -0.0002
        
Analisis: Selisih yang kecil (< 0.05) menunjukkan model dapat menggeneralisasi dengan baik (tidak overfit).
